In [49]:
import azureml.core
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig ,Environment
from azureml.core import Run, Dataset

In [ ]:
# Check the Azure ML SDK version (optional but recommended)
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [50]:
ws = Workspace.from_config()

In [ ]:
ws.datasets

In [3]:
compute_lists = ws.compute_targets

In [29]:
mlops_01 = ComputeTarget(ws,'MLOps-01')

In [33]:
mlops_01.delete()

False

In [4]:
type(compute_lists)

dict

In [13]:
compute_lists.items()

dict_items([('MLOps-01', AmlCompute(workspace=Workspace.create(name='machine-learning', subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', resource_group='machinelearning-rg'), name=MLOps-01, id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={}))])

In [16]:
import json
json.dumps(str(compute_lists),indent=4)

'"{\'MLOps-01\': AmlCompute(workspace=Workspace.create(name=\'machine-learning\', subscription_id=\'72300789-c223-4047-bfd5-2859014ad9f7\', resource_group=\'machinelearning-rg\'), name=MLOps-01, id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}"'

In [19]:
from pprint import pprint
pprint(json.dumps(str(compute_lists),indent=4))

('"{\'MLOps-01\': '
 "AmlCompute(workspace=Workspace.create(name='machine-learning', "
 "subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', "
 "resource_group='machinelearning-rg'), name=MLOps-01, "
 'id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, '
 'type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}"')


In [22]:
pprint(str(compute_lists))

("{'MLOps-01': AmlCompute(workspace=Workspace.create(name='machine-learning', "
 "subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', "
 "resource_group='machinelearning-rg'), name=MLOps-01, "
 'id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, '
 'type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}')



## Get the list of Enviornment available



In [ ]:
# Load your Azure ML workspace
ws = Workspace.from_config()

# Get a list of available curated environments
envs = Environment.list(workspace=ws)

# Print the names of available curated environments
for env_name in envs:
    print(env_name)

### Create an environemnt from the requirements txt

In [ ]:
#Create an envionment from requirements.txt file
env = Environment.from_pip_requirements(name='mlops_01_env_from_pip_local',file_path='./requirements.txt',)
# Register the enviorment to the workspace
env.register(ws) 
#build the env
env.build(ws)

# Lets Start with an Experiment

In [ ]:
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig ,Environment

# Load your Azure ML workspace from the config file
ws = Workspace.from_config()

# Create a new experiment
experiment_name = "iris-experiment"
experiment = Experiment(workspace=ws, name=experiment_name)



# Create A Run Configuration
run_config = RunConfiguration()
run_config.target = "MLOps-01"  #The compute_targets available 
run_config.environment = ws.environments['mlops_01_env_from_pip_local']



# Specify the output folder where the model and results will be saved
output_folder = "./outputs"  # Change this to your desired location


# Specify the script to run (train_01.py in this case)
script_run_config = ScriptRunConfig(source_directory=".", script="train_01.py", run_config=run_config,arguments=["--output_folder", output_folder])


# Submit the experiment
run = experiment.submit(config=script_run_config)
run.wait_for_completion(show_output=True)


Data Ingestion


In [44]:
from sklearn import datasets
import pandas as pd
# Load the dataset (e.g., Iris dataset)
iris = datasets.load_iris()

In [43]:
iris_data.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [45]:
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [47]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [51]:
# Load the dataset (e.g., Iris dataset)
iris = datasets.load_iris()

# set the iris data into a pandas df
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Register the dataset in the Azure ML workspace
iris_dataset = Dataset.Tabular.register_pandas_dataframe(dataframe=df,
                                                            target=Run.get_context().experiment.workspace,
                                                            name='iris_data',
                                                        )

AttributeError: '_OfflineRun' object has no attribute 'experiment'

In [53]:
ws.datasets

{'MD-Attrition_Model_Boosted_tree-Train_Model-Trained_model-d76aed3e': DatasetRegistration(id='fab85361-fe50-4a90-a568-5d47fd9fd01e', name='MD-Attrition_Model_Boosted_tree-Train_Model-Trained_model-d76aed3e', version=1, description='This is a dataset promoted by inference graph generation automatically on 12/26/2022 09:06:00. 

 Training pipeline run name:Attrition_Model Boosted_tree 

Training pipeline run id:6f1e2277-3982-437f-aeb2-7548630217ad 

Module Name:Train Model 

ModuleId:561d2c00-323e-50eb-afcf-acc6694e3244 

Port:Trained_model', tags={'azureml.Designer': 'true'}), 'attrition_data': DatasetRegistration(id='148931a4-700a-4021-a40a-8b2d74f3b881', name='attrition_data', version=2, description='', tags={})}